In [7]:
import pandas as pd
import sklearn 
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import numpy as np
from sklearn import feature_selection
import seaborn as sns
import openpyxl

In [14]:
janela = 'diaria'
book_name='bookx'
df = pd.read_csv("P:\\ciencia_de_dados\\Correlacao_de_fundos\\Bases\\"  + book_name + "_" + janela + ".csv",delimiter=',',decimal='.',parse_dates=True)
df = df.dropna().drop_duplicates().rename(columns={'Unnamed: 0':'data'})
df.head()

,data,JPEIGLSP Index,BRL Curncy,CAD Curncy,AUD Curncy,DXY Index
2,2021-10-20,322.75,5.5684,1.2363,0.7474,93.734
3,2021-10-19,322.75,5.5847,1.2379,0.7411,93.953
4,2021-10-18,328.24,5.5127,1.2368,0.7421,93.937
5,2021-10-15,325.05,5.4608,1.237,0.7416,93.956
6,2021-10-14,330.46,5.5126,1.2443,0.7379,94.08


In [15]:
df = df.melt(id_vars='data', value_vars=list(df.columns[1:]), value_name="FinancialPrice",var_name='Product')

In [16]:
#Calculo das colunas temporais
df['data']=pd.to_datetime(df['data'],format = '%Y-%m-%d')
df['MesAno'] = df['data'].dt.strftime('%m-%Y')
df['Mes_nomial']=df['data'].dt.strftime('%b')
df['Mes']=df['data'].dt.strftime('%m').astype(int)
df['Ano']=df['data'].dt.strftime('%Y').astype(int)

In [18]:
df

,data,Product,FinancialPrice,MesAno,Mes_nomial,Mes,Ano
0,2021-10-20,JPEIGLSP Index,322.75,10-2021,Oct,10,2021
1,2021-10-19,JPEIGLSP Index,322.75,10-2021,Oct,10,2021
2,2021-10-18,JPEIGLSP Index,328.24,10-2021,Oct,10,2021
3,2021-10-15,JPEIGLSP Index,325.05,10-2021,Oct,10,2021
4,2021-10-14,JPEIGLSP Index,330.46,10-2021,Oct,10,2021
...,...,...,...,...,...,...,...
15100,2010-03-30,DXY Index,81.358,03-2010,Mar,3,2010
15101,2010-03-29,DXY Index,81.676,03-2010,Mar,3,2010
15102,2010-03-26,DXY Index,82.12,03-2010,Mar,3,2010
15103,2010-03-25,DXY Index,81.838,03-2010,Mar,3,2010


In [28]:
if janela == 'diaria':
    df = df.groupby(['Product','MesAno']).agg(['max'])
    df.columns = df.columns.droplevel(1)
    df = df1.reset_index()
    df.head()

In [29]:
df.sort_values(by=["Product",'data'],ascending=False,inplace=True)

In [30]:
df

,index,Product,MesAno,data,FinancialPrice,Mes_nomial,Mes,Ano
677,677,JPEIGLSP Index,10-2021,2021-10-20,333,Oct,10,2021
665,665,JPEIGLSP Index,09-2021,2021-09-30,324.37,Sep,9,2021
653,653,JPEIGLSP Index,08-2021,2021-08-31,327.44,Aug,8,2021
641,641,JPEIGLSP Index,07-2021,2021-07-30,331.83,Jul,7,2021
629,629,JPEIGLSP Index,06-2021,2021-06-30,312.5,Jun,6,2021
...,...,...,...,...,...,...,...,...
70,70,AUD Curncy,07-2010,2010-07-30,0.9026,Jul,7,2010
58,58,AUD Curncy,06-2010,2010-06-30,0.8765,Jun,6,2010
46,46,AUD Curncy,05-2010,2010-05-31,0.9261,May,5,2010
34,34,AUD Curncy,04-2010,2010-04-30,0.9351,Apr,4,2010


In [31]:
#Cria a coluna retorno e seta NaN na primeira linha de cada fundo
df['FinancialPrice']=pd.to_numeric(df["FinancialPrice"], downcast="float")
df["Retorno_1"] = (df.FinancialPrice.shift())/(df.FinancialPrice)
df.loc[df.drop_duplicates('Product').index.values,'Retorno_1'] = np.NaN
df['Retorno'] = df.Retorno_1.shift(periods=-1)
df['Retorno'] = (df["Retorno_1"]-1)

In [32]:
df.to_csv('P:\\ciencia_de_dados\\Correlacao_de_fundos\\Desenvolvimento\\1DataPrep\\base_'+ book_name + '.csv',sep =';',decimal=',')